# QLoRA Fine-Tuning for LLM Unlearning - Yelp Dataset (Local Machine)

This notebook demonstrates how to fine-tune a large language model using QLoRA (Quantized Low-Rank Adaptation) for sentiment analysis tasks with unlearning capabilities using the **Yelp Polarity dataset**.

**Key Features:**
- Uses pre-split Yelp dataset from HuggingFace (by research paper authors)
- Dataset already has forget/retain splits created via k-means clustering
- Optimized for local machine execution

## Setup Steps:
1. Install required dependencies
2. Import libraries and define utilities
3. Load pre-split dataset from HuggingFace
4. Load model and evaluate base model (0-shot)
5. Train the model on the dataset
6. Evaluate fine-tuned model
7. Save the fine-tuned model

## 1. Install Dependencies

Install all required packages including sentence-transformers for embeddings.

In [ ]:
# Install dependencies - uncomment if needed
# !pip install -q datasets evaluate peft scikit-learn
# !pip install -q torch torchinfo transformers trl
# !pip install -q bitsandbytes wandb
# !pip install -q accelerate

print("✅ Dependencies should be installed. If not, uncomment the pip install commands above.")

## 2. Import Required Libraries

In [ ]:
import os
import sys
import time
import pickle
import datetime
import random
from copy import deepcopy
from argparse import Namespace

import numpy as np
from tqdm import tqdm
import evaluate

import torch
from torch.utils.data import DataLoader
from torchinfo import summary

from datasets import load_dataset, concatenate_datasets, Dataset, DatasetDict
from peft import get_peft_model, LoraConfig, TaskType, PeftConfig, PeftModel
from transformers import AutoTokenizer, TrainerState, TrainerControl, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import TrainingArguments, TrainerCallback, Trainer
from trl import SFTTrainer, SFTConfig

print("All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

In [ ]:
# Clear GPU memory if needed
import torch
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("✅ GPU cache cleared")
    print(f"GPU Memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"GPU Memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")
else:
    print("⚠️  No CUDA available")

In [ ]:
# Set CUDA environment for better error reporting
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
print("✅ CUDA_LAUNCH_BLOCKING enabled for better error reporting")

In [ ]:
# Check PyTorch CUDA compatibility
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version (PyTorch): {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    
    # Try a simple CUDA operation
    try:
        x = torch.randn(10, 10).cuda()
        print("✅ CUDA test operation successful!")
        del x
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"❌ CUDA test failed: {e}")

## 3. Define Utility Functions

These functions are adapted from `utils.py` with Yelp dataset support.

In [ ]:
def get_data_path(dataset):
    """Get the HuggingFace dataset path based on dataset name."""
    if dataset.lower() == "sst2":
        data_path = "karuna-bhaila/Unlearning_SST2v3"
    elif dataset.lower() == 'yelp':
        data_path = "karuna-bhaila/Unlearning_Yelp_Polarity"
    else:
        # define dataset with the following splits:
        # train_retain, train_forget, test_retain, test_forget
        raise NotImplementedError(f"Dataset {dataset} not supported")

    return data_path


def preprocess_logits_for_metrics(logits, labels):
    """Preprocess logits for metric computation."""
    if isinstance(logits, tuple):
        # Depending on the model and config, logits may contain extra tensors,
        # like past_key_values, but logits always come first
        logits = logits[0]
    # argmax to get the token ids
    return logits.argmax(dim=-1)


def compute_metrics(eval_pred):
    """Compute evaluation metrics."""
    f1_metric = evaluate.load("f1")
    accuracy_metric = evaluate.load("accuracy")
    precision_metric = evaluate.load('precision')
    recall_metric = evaluate.load('recall')

    logits, labels = eval_pred

    predictions = logits[:, :-1]
    labels = labels[:, 1:]

    check_labels = labels != -100

    last_token_predictions = []
    last_token_labels = []

    for idx in range(len(predictions)):
        last_token_predictions.append(predictions[idx][check_labels[idx]])
        last_token_labels.append(labels[idx][check_labels[idx]])

    f1 = f1_metric.compute(predictions=last_token_predictions, references=last_token_labels, average='weighted')["f1"]
    accuracy = accuracy_metric.compute(predictions=last_token_predictions, references=last_token_labels)["accuracy"]
    precision = precision_metric.compute(predictions=last_token_predictions, references=last_token_labels, average='micro')['precision']
    recall = recall_metric.compute(predictions=last_token_predictions, references=last_token_labels, average='micro')['recall']

    return {"f1-score": f1, 'accuracy': accuracy, 'precision': precision, 'recall': recall}


class CustomCallback(TrainerCallback):
    """Custom callback for additional evaluation during training."""
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer

    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy

    def on_evaluate(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.eval_dataset['train_retain'],
                                   metric_key_prefix="eval_train_retrain")
            self._trainer.evaluate(eval_dataset=self._trainer.eval_dataset['train_forget'],
                                   metric_key_prefix="eval_train_forget")
            return control_copy

print("Utility functions defined successfully!")

## 4. Define Model and Dataset Functions

These functions handle model initialization and dataset preparation.

In [ ]:
def get_lora_model(model_checkpoints, rank=4, alpha=16, lora_dropout=0.1, bias='none'):
    """Initialize model with QLoRA configuration."""
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=model_checkpoints,
        device_map="auto",
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # Configure LoRA with proper target modules and NO embedding/head training
    if model_checkpoints == 'mistralai/Mistral-7B-v0.1' or model_checkpoints == 'meta-llama/Llama-2-7b-hf':
        peft_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            r=rank,
            lora_alpha=alpha,
            lora_dropout=lora_dropout,
            bias=bias,
            target_modules=[
                "q_proj",
                "v_proj",
            ],
            inference_mode=False,  # Training mode
            modules_to_save=None,  # Don't train embeddings or LM head
        )
    else:
        # For OPT models, explicitly target attention layers only
        peft_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            r=rank,
            lora_alpha=alpha,
            lora_dropout=lora_dropout,
            bias=bias,
            target_modules=["q_proj", "v_proj", "k_proj", "out_proj"],  # Explicit target modules for OPT
            inference_mode=False,  # Training mode
            modules_to_save=None,  # Don't train embeddings or LM head
        )

    return model, tokenizer, peft_config


def get_unlearn_dataset(
        data_path,
        tokenizer,
        add_prefix_space=True,
        max_length=1024,
        truncation=True
):
    """Load and prepare the unlearning dataset."""
    # Format: prompt and completion for completion-only loss
    # The model will only compute loss on the completion part
    def _preprocessing_sentiment(examples):
        prompt = f"""### Text: {examples['text']}\n\n### Question: What is the sentiment of the given text?\n\n### Sentiment:"""
        completion = f""" {examples['label_text']}"""
        # Combine for SFTTrainer with completion_only_loss
        return {"text": prompt + completion}

    data = load_dataset(data_path)

    data = data.map(_preprocessing_sentiment, batched=False)
    data = data.remove_columns(['label', 'label_text'])
    data.set_format("torch")

    print(data)

    return data

print("Model and dataset functions defined successfully!")

## 7. Configure Weights & Biases (Optional)

If you want to track your training with W&B, login here. Otherwise, disable W&B tracking.

In [ ]:
import wandb
import os

# Disable W&B tracking (uncomment wandb.login() to enable)
os.environ["WANDB_DISABLED"] = "true"

# To enable W&B tracking:
# wandb.login()
# del os.environ["WANDB_DISABLED"]

print("W&B configuration set!")
print(f"W&B tracking: {'disabled' if os.environ.get('WANDB_DISABLED') == 'true' else 'enabled'}")

## 5. Configure Training Parameters

Modify these parameters according to your needs.

In [ ]:
# Create arguments namespace (similar to command-line arguments)
args = Namespace(
    # Dataset configuration
    dataset="yelp",  # Changed from 'sst2' to 'yelp'

    # Model configuration
    model_name="facebook/opt-1.3b",  # OPT-1.3B model
    # Alternative models:
    # - "meta-llama/Llama-2-7b-hf"
    # - "meta-llama/Llama-2-13b-hf"

    # Output configuration
    output_path=None,  # Will be auto-generated if None

    # Training hyperparameters
    max_length=1024,
    lr=1e-4,
    train_batch_size=45,  # Adjust based on your GPU memory
    eval_batch_size=45,
    num_epochs=5,
    weight_decay=0.001,

    # LoRA configuration
    lora_rank=16,
    lora_alpha=64,
    lora_dropout=0.1,
    lora_bias='none',  # Options: 'lora_only', 'none', 'all'

    # Model-specific settings
    set_pad_id=False,  # Set to True for Mistral-7B
)

print("Configuration:")
print(f"  Dataset: {args.dataset}")
print(f"  Model: {args.model_name}")
print(f"  Learning Rate: {args.lr}")
print(f"  Batch Size: {args.train_batch_size}")
print(f"  Epochs: {args.num_epochs}")
print(f"  LoRA Rank: {args.lora_rank}")
print(f"  LoRA Alpha: {args.lora_alpha}")

## 6. Setup Output Directory and W&B Project

In [ ]:
# Determine model name for output directory
if 'llama-2-7b' in args.model_name.lower():
    model_name = 'llama-2-7b-hf'
elif 'llama-2-13b' in args.model_name.lower():
    model_name = 'llama-2-13b-hf'
elif 'opt-1.3b' in args.model_name.lower():
    model_name = 'opt-1.3b'
elif 'mistral' in args.model_name.lower():
    model_name = 'mistral-7b'
else:
    model_name = 'custom-model'

# Setup W&B project (if enabled)
if os.environ.get("WANDB_DISABLED") != "true":
    os.environ["WANDB_LOG_MODEL"] = "all"
    os.environ["WANDB_PROJECT"] = f'qlora_{model_name.lower()}_{args.dataset.lower()}'

# Get data path for Yelp
data_path = get_data_path(args.dataset)

# Setup output directory
if args.output_path is None:
    args.output_path = f'qlora_checkpoints/{model_name.lower()}-qlora-{args.dataset.lower()}'

os.makedirs(args.output_path, exist_ok=True)

# Save arguments
with open(os.path.join(args.output_path, 'arguments.txt'), 'w') as f:
    for k, v in args.__dict__.items():
        f.write(f'{k}: {v}\n')

print(f"Output directory: {args.output_path}")
print(f"Data path: {data_path}")

## 7. Initialize Model and Tokenizer

**Note:** For local machine, HuggingFace token may be required for some models.

In [ ]:
# Optional: Login to Hugging Face Hub if needed for gated models
# from huggingface_hub import login
# hf_token = "your_hf_token_here"  # Replace with your token
# login(hf_token)

print("Loading model and tokenizer...")
print("This may take a few minutes...")

model, tokenizer, lora_config = get_lora_model(
    args.model_name,
    rank=args.lora_rank,
    alpha=args.lora_alpha,
    lora_dropout=args.lora_dropout,
    bias=args.lora_bias
)

# CRITICAL: Explicitly freeze ALL base model parameters before applying LoRA
# This prevents SFTTrainer from making embeddings/layer norms trainable
print("\n🔒 Freezing base model parameters...")
for name, param in model.named_parameters():
    param.requires_grad = False  # Freeze everything first

print("✅ All base model parameters frozen!")

# Now apply LoRA using get_peft_model - this will ONLY make LoRA params trainable
model = get_peft_model(model, lora_config)

print("✅ LoRA adapters added!")
print("\nModel loaded successfully!")
print("\nModel Summary:")
summary(model)

## 8. Load Pre-Split Yelp Dataset from HuggingFace

Load the Yelp dataset that has already been split into train/test retain/forget by the research paper authors.

In [ ]:
print("Loading Yelp dataset...")

dataset = get_unlearn_dataset(
    data_path,
    tokenizer=tokenizer,
    max_length=args.max_length,
    add_prefix_space=True,
    truncation=True,
)

print("\nYelp Dataset splits:")
for split in dataset.keys():
    print(f"  {split}: {len(dataset[split])} examples")

# Show a sample from each split
print("\n📝 Sample from each split:")
for split in dataset.keys():
    print(f"\n{split} example:")
    print(dataset[split]['text'][0][:200] + "...")

## 9. Verify LoRA Configuration

**IMPORTANT:** Check that only LoRA adapters are trainable, not the full model!

In [ ]:
# Verify LoRA parameters - should be ~1-4M trainable params, not 200M+!
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    trainable_percentage = 100 * trainable_params / all_param

    print(f"╔═══════════════════════════════════════════════╗")
    print(f"║          LoRA Parameter Verification         ║")
    print(f"╠═══════════════════════════════════════════════╣")
    print(f"║ Total params:      {all_param:>15,} ║")
    print(f"║ Trainable params:  {trainable_params:>15,} ║")
    print(f"║ Non-trainable:     {all_param - trainable_params:>15,} ║")
    print(f"║ Trainable %:       {trainable_percentage:>14.4f}% ║")
    print(f"╚═══════════════════════════════════════════════╝")

    # Validation checks
    if trainable_percentage > 5.0:
        print("\n⚠️  WARNING: Trainable params > 5%! This is NOT typical for LoRA!")
        print("   Expected: 0.1-1% for LoRA (rank 8-64)")
        print("   Check that embeddings/LM head are NOT being trained.")
    elif trainable_percentage < 0.05:
        print("\n⚠️  WARNING: Trainable params < 0.05%! This might be too few.")
        print("   Consider increasing LoRA rank or checking target_modules.")
    else:
        print("\n✅ Parameter count looks good for LoRA training!")

    return trainable_params, all_param

# Print detailed parameter info
print("\n" + "="*60)
trainable, total = print_trainable_parameters(model)
print("="*60)

# Print which modules are trainable
print("\n📋 Trainable modules:")
trainable_modules = []
for name, param in model.named_parameters():
    if param.requires_grad:
        trainable_modules.append(name)

if len(trainable_modules) <= 50:  # Only print if reasonable number
    for name in trainable_modules:
        print(f"  ✓ {name}")
else:
    print(f"  Total: {len(trainable_modules)} trainable modules")
    print("  Sample (first 10):")
    for name in trainable_modules[:10]:
        print(f"  ✓ {name}")

In [ ]:
print("Evaluating base model before fine-tuning...")
print("This will give us a baseline to compare against.")

from torch.utils.data import DataLoader
from tqdm import tqdm

def evaluate_base_model_simple(model, dataset, tokenizer, batch_size=8):
    """
    Simple evaluation that computes accuracy by checking if the model
    generates the correct sentiment token.
    """
    model.eval()

    correct = 0
    total = 0

    # Map sentiment labels to expected tokens
    sentiment_mapping = {
        'positive': tokenizer.encode(' positive', add_special_tokens=False)[0],
        'negative': tokenizer.encode(' negative', add_special_tokens=False)[0],
        'Positive': tokenizer.encode(' Positive', add_special_tokens=False)[0],
        'Negative': tokenizer.encode(' Negative', add_special_tokens=False)[0],
    }

    with torch.no_grad():
        for i in tqdm(range(0, len(dataset), batch_size), desc="Evaluating"):
            batch_end = min(i + batch_size, len(dataset))
            batch = dataset[i:batch_end]

            for example in batch['text']:
                # Split into prompt and completion
                if "### Sentiment:" in example:
                    prompt_part = example.split("### Sentiment:")[0] + "### Sentiment:"
                    true_sentiment = example.split("### Sentiment:")[1].strip()

                    # Tokenize prompt
                    inputs = tokenizer(prompt_part, return_tensors="pt", truncation=True, max_length=512)
                    inputs = {k: v.to(model.device) for k, v in inputs.items()}

                    # Generate next token
                    outputs = model(**inputs)
                    logits = outputs.logits
                    next_token_logits = logits[0, -1, :]
                    predicted_token = torch.argmax(next_token_logits).item()

                    # Check if predicted token matches any sentiment token
                    true_sentiment_lower = true_sentiment.lower().strip()

                    # Get expected token(s)
                    expected_tokens = []
                    if 'positive' in true_sentiment_lower:
                        expected_tokens = [sentiment_mapping.get('positive'), sentiment_mapping.get('Positive')]
                    elif 'negative' in true_sentiment_lower:
                        expected_tokens = [sentiment_mapping.get('negative'), sentiment_mapping.get('Negative')]

                    expected_tokens = [t for t in expected_tokens if t is not None]

                    if predicted_token in expected_tokens:
                        correct += 1

                    total += 1

    accuracy = correct / total if total > 0 else 0.0
    return accuracy

print("\n📊 Base Model Evaluation Results (0-shot on Yelp):")
print("="*60)

# Evaluate on all 4 splits: train_retain, train_forget, test_retain, test_forget
print("\n🔹 Evaluating Train Retain (Base Model)...")
train_retain_acc = evaluate_base_model_simple(model, dataset['train_retain'], tokenizer, batch_size=8)
print(f"  Accuracy: {train_retain_acc:.4f}")

print("\n🔹 Evaluating Train Forget (Base Model)...")
train_forget_acc = evaluate_base_model_simple(model, dataset['train_forget'], tokenizer, batch_size=8)
print(f"  Accuracy: {train_forget_acc:.4f}")

print("\n🔹 Evaluating Test Retain (Base Model)...")
test_retain_acc = evaluate_base_model_simple(model, dataset['test_retain'], tokenizer, batch_size=8)
print(f"  Accuracy: {test_retain_acc:.4f}")

print("\n🔹 Evaluating Test Forget (Base Model)...")
test_forget_acc = evaluate_base_model_simple(model, dataset['test_forget'], tokenizer, batch_size=8)
print(f"  Accuracy: {test_forget_acc:.4f}")

print("="*60)

# Store baseline results for comparison later (4 splits)
baseline_results = {
    'train_retain': {'accuracy': train_retain_acc},
    'train_forget': {'accuracy': train_forget_acc},
    'test_retain': {'accuracy': test_retain_acc},
    'test_forget': {'accuracy': test_forget_acc}
}

# Save baseline results
baseline_path = os.path.join(args.output_path, 'baseline_results.pkl')
with open(baseline_path, 'wb') as f:
    pickle.dump(baseline_results, f)

print("\n✅ Base model evaluation complete!")
print(f"Baseline results saved to: {baseline_path}")
print(f"Note: Low accuracy (~1-5%) is expected for zero-shot on untrained base models")

## 11. Save Dataset Splits Locally

Save the preprocessed dataset splits for later use.

In [ ]:
print("Saving dataset splits locally...")

# Create datasets directory in output path
datasets_dir = os.path.join(args.output_path, 'datasets')
os.makedirs(datasets_dir, exist_ok=True)

# Save each split
for split_name in dataset.keys():
    dataset[split_name].save_to_disk(os.path.join(datasets_dir, split_name))
    print(f"  ✓ Saved {split_name}: {len(dataset[split_name])} examples")

print(f"\n✅ All dataset splits saved to: {datasets_dir}")
print("\nTo load later, use:")
print("  from datasets import load_from_disk")
print(f"  dataset = load_from_disk('{datasets_dir}/train_retain')")

## 12. Configure Training Arguments

Using SFTConfig with completion_only_loss for modern TRL approach.

In [ ]:
# Use SFTConfig instead of TrainingArguments for modern TRL
# This includes completion_only_loss=True to mask loss on prompt tokens
training_args = SFTConfig(
    output_dir=args.output_path,
    learning_rate=args.lr,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    per_device_train_batch_size=args.train_batch_size,
    per_device_eval_batch_size=args.eval_batch_size,
    num_train_epochs=args.num_epochs,
    weight_decay=args.weight_decay,
    eval_strategy="no",
    save_strategy="no",
    load_best_model_at_end=True,
    gradient_checkpointing=True,
    fp16=True,
    report_to="wandb" if os.environ.get("WANDB_DISABLED") != "true" else "none",
    run_name=f'lr={args.lr}',
    max_grad_norm=0.3,
    metric_for_best_model="eval_test_loss",
    logging_steps=100,
    # SFT-specific parameters
    max_seq_length=args.max_length,
    dataset_text_field='text',
    packing=False,  # Disable packing for clearer training
)

print("Training arguments configured with SFTConfig!")
print("Using modern TRL approach optimized for local machine execution")

## 13. Set Padding Token (if needed)

In [ ]:
if args.set_pad_id:
    model.config.pad_token_id = model.config.eos_token_id
    print("Padding token ID set to EOS token ID")

# Ensure model is on GPU
if torch.cuda.is_available():
    if model.device.type != 'cuda':
        model = model.to('cuda')
        print("Model moved to GPU")
    else:
        print("Model already on GPU")
else:
    print("⚠️  No GPU detected. Training will be VERY slow on CPU.")
    print("   Consider using a machine with CUDA-enabled GPU.")

## 14. Initialize Trainer

In [ ]:
print("Initializing trainer...")

# Initialize SFTTrainer with the model that already has LoRA adapters
trainer = SFTTrainer(
    model=model,  # Already has LoRA adapters applied
    args=training_args,  # SFTConfig
    peft_config=lora_config,
    processing_class=tokenizer,
    train_dataset=concatenate_datasets([dataset['train_retain'], dataset['train_forget']]),
    eval_dataset={"test": concatenate_datasets([dataset['test_retain'], dataset['test_forget']])},
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

# Add custom callback for additional evaluation
trainer.add_callback(CustomCallback(trainer))

print("Trainer initialized successfully!")
print(f"Training on combined dataset: {len(trainer.train_dataset)} examples")
print(f"  - Train Retain: {len(dataset['train_retain'])} examples")
print(f"  - Train Forget: {len(dataset['train_forget'])} examples")

## 15. Train the Model

This will start the training process. Training time depends on your GPU and dataset size.

In [ ]:
print("Starting training on Yelp dataset...")
print(f"Training for {args.num_epochs} epochs")
print("="*60)

start_time = time.perf_counter()

trainer.train()

runtime = time.perf_counter() - start_time

print("="*60)
print(f"✅ Training completed!")
print(f"Total training time: {runtime:.2f} seconds ({runtime/60:.2f} minutes)")

## 16. Evaluate Fine-tuned Model on All Splits

Evaluate the fine-tuned model on all 4 splits and compare with baseline.

In [ ]:
print("="*60)
print("📊 Fine-tuned Model Evaluation Results:")
print("="*60)

def evaluate_finetuned_model(model, dataset, tokenizer, batch_size=8, split_name=""):
    """
    Evaluate fine-tuned model using the same approach as base model evaluation.
    Returns accuracy only (matching baseline format).
    """
    model.eval()
    
    correct = 0
    total = 0
    
    # Map sentiment labels to expected tokens
    sentiment_mapping = {
        'positive': tokenizer.encode(' positive', add_special_tokens=False)[0],
        'negative': tokenizer.encode(' negative', add_special_tokens=False)[0],
        'Positive': tokenizer.encode(' Positive', add_special_tokens=False)[0],
        'Negative': tokenizer.encode(' Negative', add_special_tokens=False)[0],
    }
    
    with torch.no_grad():
        for i in tqdm(range(0, len(dataset), batch_size), desc=f"Evaluating {split_name}"):
            batch_end = min(i + batch_size, len(dataset))
            batch = dataset[i:batch_end]
            
            for example in batch['text']:
                # Split into prompt and completion
                if "### Sentiment:" in example:
                    prompt_part = example.split("### Sentiment:")[0] + "### Sentiment:"
                    true_sentiment = example.split("### Sentiment:")[1].strip()
                    
                    # Tokenize prompt
                    inputs = tokenizer(prompt_part, return_tensors="pt", truncation=True, max_length=512)
                    inputs = {k: v.to(model.device) for k, v in inputs.items()}
                    
                    # Get model prediction
                    outputs = model(**inputs)
                    logits = outputs.logits
                    next_token_logits = logits[0, -1, :]
                    predicted_token = torch.argmax(next_token_logits).item()
                    
                    # Check if predicted token matches any sentiment token
                    true_sentiment_lower = true_sentiment.lower().strip()
                    
                    # Get expected token(s)
                    expected_tokens = []
                    if 'positive' in true_sentiment_lower:
                        expected_tokens = [sentiment_mapping.get('positive'), sentiment_mapping.get('Positive')]
                    elif 'negative' in true_sentiment_lower:
                        expected_tokens = [sentiment_mapping.get('negative'), sentiment_mapping.get('Negative')]
                    
                    expected_tokens = [t for t in expected_tokens if t is not None]
                    
                    if predicted_token in expected_tokens:
                        correct += 1
                    
                    total += 1
    
    accuracy = correct / total if total > 0 else 0.0
    return {'accuracy': accuracy}

# Evaluate on all 4 splits
print("\n🔹 Evaluating Train Retain (Fine-tuned Model)...")
train_retain_finetuned = evaluate_finetuned_model(model, dataset['train_retain'], tokenizer, batch_size=8, split_name="Train Retain")
print(f"  Accuracy: {train_retain_finetuned['accuracy']:.4f}")

print("\n🔹 Evaluating Train Forget (Fine-tuned Model)...")
train_forget_finetuned = evaluate_finetuned_model(model, dataset['train_forget'], tokenizer, batch_size=8, split_name="Train Forget")
print(f"  Accuracy: {train_forget_finetuned['accuracy']:.4f}")

print("\n🔹 Evaluating Test Retain (Fine-tuned Model)...")
test_retain_finetuned = evaluate_finetuned_model(model, dataset['test_retain'], tokenizer, batch_size=8, split_name="Test Retain")
print(f"  Accuracy: {test_retain_finetuned['accuracy']:.4f}")

print("\n🔹 Evaluating Test Forget (Fine-tuned Model)...")
test_forget_finetuned = evaluate_finetuned_model(model, dataset['test_forget'], tokenizer, batch_size=8, split_name="Test Forget")
print(f"  Accuracy: {test_forget_finetuned['accuracy']:.4f}")

print("="*60)

# Store fine-tuned results
finetuned_results = {
    'train_retain': train_retain_finetuned,
    'train_forget': train_forget_finetuned,
    'test_retain': test_retain_finetuned,
    'test_forget': test_forget_finetuned
}

# Compare with baseline
print("\n" + "="*60)
print("📈 Comparison: Base vs Fine-tuned Model")
print("="*60)

def compare_results(split_name, baseline, finetuned):
    """Compare baseline and fine-tuned results for a split."""
    base_acc = baseline['accuracy']
    ft_acc = finetuned['accuracy']
    improvement = ft_acc - base_acc
    
    print(f"\n🔹 {split_name}:")
    print(f"   Base Model:      {base_acc:.4f}")
    print(f"   Fine-tuned:      {ft_acc:.4f}")
    print(f"   Improvement:     {improvement:+.4f}")
    
    if improvement > 0:
        print(f"   Status:          ✅ Improved")
    elif improvement < 0:
        print(f"   Status:          ⚠️  Degraded")
    else:
        print(f"   Status:          ➡️  No change")

compare_results("Train Retain", baseline_results['train_retain'], train_retain_finetuned)
compare_results("Train Forget", baseline_results['train_forget'], train_forget_finetuned)
compare_results("Test Retain", baseline_results['test_retain'], test_retain_finetuned)
compare_results("Test Forget", baseline_results['test_forget'], test_forget_finetuned)

print("\n" + "="*60)

# Save comparison results
comparison_path = os.path.join(args.output_path, 'evaluation_comparison.pkl')
with open(comparison_path, 'wb') as f:
    pickle.dump({
        'baseline': baseline_results,
        'finetuned': finetuned_results
    }, f)

print(f"✅ Evaluation comparison saved to: {comparison_path}")

## 17. Save the Fine-tuned Model

Save the model, tokenizer, and all training artifacts.

In [ ]:
# Save the fine-tuned model locally
print("Saving model locally...")

# Save to local output directory
final_model_path = os.path.join(args.output_path, 'final_model')
trainer.save_model(final_model_path)

# Save tokenizer
tokenizer.save_pretrained(final_model_path)

# Save arguments
with open(os.path.join(final_model_path, 'arguments.txt'), 'w') as f:
    for k, v in args.__dict__.items():
        f.write(f'{k}: {v}\n')

# Copy evaluation results to final model directory
import shutil
eval_comparison_path = os.path.join(args.output_path, 'evaluation_comparison.pkl')
if os.path.exists(eval_comparison_path):
    shutil.copy(eval_comparison_path, os.path.join(final_model_path, 'evaluation_comparison.pkl'))
    print("Evaluation comparison copied to model directory.")

baseline_path = os.path.join(args.output_path, 'baseline_results.pkl')
if os.path.exists(baseline_path):
    shutil.copy(baseline_path, os.path.join(final_model_path, 'baseline_results.pkl'))
    print("Baseline results copied to model directory.")

print(f"\n✅ Model saved successfully!")
print(f"  Location: {final_model_path}")
print(f"\nModel files saved:")
print(f"  - LoRA adapters (adapter_model.safetensors)")
print(f"  - Tokenizer files")
print(f"  - Training arguments")
print(f"  - Evaluation results")
print(f"  - Baseline results")
print("\n🎉 Training complete! You can now use this model for unlearning experiments.")

## 18. Test with Custom Examples (Optional)

Try the model with your own text samples to see how it performs.

In [ ]:
def predict_sentiment(text, model, tokenizer):
    """
    Predict sentiment for a given text.
    
    Args:
        text: Input text to analyze
        model: Fine-tuned model
        tokenizer: Tokenizer
        
    Returns:
        predicted_sentiment: The predicted sentiment label
    """
    # Format prompt exactly as during training
    prompt = f"""### Text: {text}\n\n### Question: What is the sentiment of the given text?\n\n### Sentiment:"""
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # Get prediction
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        next_token_logits = logits[0, -1, :]
        predicted_token = torch.argmax(next_token_logits).item()
    
    # Decode token
    predicted_text = tokenizer.decode([predicted_token])
    
    # Map to sentiment
    if 'positive' in predicted_text.lower():
        return 'positive', predicted_text
    elif 'negative' in predicted_text.lower():
        return 'negative', predicted_text
    else:
        return 'unknown', predicted_text

# Test examples (Yelp-style reviews)
test_examples = [
    "This restaurant was absolutely amazing! The food was delicious and the service was excellent.",
    "Terrible experience. The food was cold and the staff was rude. Will never come back.",
    "The product arrived damaged and customer service was unhelpful. Very disappointed.",
    "Best hotel I've ever stayed at! The room was clean and the staff was incredibly friendly.",
    "Decent place, nothing extraordinary. The food was okay but overpriced.",
]

print("="*60)
print("🎯 Testing with Custom Examples (Yelp-style reviews)")
print("="*60)

for i, text in enumerate(test_examples, 1):
    sentiment, token = predict_sentiment(text, model, tokenizer)
    print(f"\nExample {i}:")
    print(f"Review: {text}")
    print(f"Predicted Sentiment: {sentiment} (token: {token!r})")
    print("-"*60)

print("\n✅ Custom testing complete!")

# Try your own text
print("\n" + "="*60)
print("💡 To try your own text, modify and run:")
print("="*60)
print('your_text = "Your custom review or text here"')
print('sentiment, token = predict_sentiment(your_text, model, tokenizer)')
print('print(f"Predicted: {sentiment} (token: {token!r})")')
print("="*60)

---

## Summary and Next Steps

### ✅ What We Did:
1. ✅ Loaded **pre-split Yelp dataset** from HuggingFace (`karuna-bhaila/Unlearning_Yelp_Polarity`)
   - Dataset already has forget/retain splits created by research paper authors
2. ✅ Configured QLoRA fine-tuning for **OPT-1.3B** on the pre-split dataset
3. ✅ Evaluated base model (0-shot) on all 4 splits:
   - train_retain, train_forget, test_retain, test_forget
4. ✅ Fine-tuned the model on combined train_retain + train_forget
5. ✅ Evaluated fine-tuned model on all 4 splits
6. ✅ Compared baseline vs fine-tuned performance
7. ✅ Saved model, datasets, and evaluation results locally

### 📊 Dataset Details:
- **Source**: `karuna-bhaila/Unlearning_Yelp_Polarity` (HuggingFace)
- **Pre-split by**: Research paper authors (likely using k-means clustering)
- **Splits**: train_retain, train_forget, test_retain, test_forget

### 📂 Evaluation Splits:
- **train_retain**: Training data from retain set (model should remember)
- **train_forget**: Training data from forget set (for unlearning)
- **test_retain**: Test data from retain set (retained knowledge)
- **test_forget**: Test data from forget set (forgotten knowledge)

### 🎯 For Unlearning Experiments:
Now that the model is fine-tuned on the full dataset, you can:
1. Apply unlearning techniques to "forget" the train_forget examples
2. Evaluate on test_forget to measure forgetting
3. Evaluate on test_retain to ensure retained knowledge is preserved
4. Compare metrics to determine unlearning effectiveness

### 📂 Saved Artifacts:
- **Model**: `{args.output_path}/final_model/`
- **Formatted datasets**: `{args.output_path}/datasets/`
- **Baseline**: `{args.output_path}/baseline_results.pkl`
- **Comparison**: `{args.output_path}/evaluation_comparison.pkl`

### 🚀 Next Steps:
1. Implement unlearning algorithm (e.g., gradient ascent on train_forget)
2. Evaluate unlearning effectiveness using test_retain and test_forget
3. Measure metrics: accuracy, F1-score, precision, recall
4. Compare forgetting rate vs retention rate

**Good luck with your unlearning experiments! 🎉**